# Verkennende analyse Ames Housing dataset

In dit notebook doen we een verkennende analyse op een subset van de Ames Housing dataset. De Ames Housing dataset bevat data over huizen in Boston, met name ook de verkoopprijs. Het is een aangepaste versie van een dataset die vaak gebruikt wordt om regressiemodellen (om de verkoopprijs mee te voorspellen) te benchmarken.

De dataset kun je vinden op Kaggle: [https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data).

Het doel van de verkennende analyse is om een eerste inzicht in de data te krijgen zonder bezig te zijn met het beantwoorden van een specifieke onderzoeksvraag. Wel houden we in ons achterhoofd dat we de dataset mogelijk willen gebruiken voor machine learning.

We zetten de verkennende analyse systematisch op door naar de volgende onderwerpen te kijken:
- Data en data dictionary
- De afzonderlijke variabelen
- Relaties tussen de variabelen

We beginnen zoals gebruikelijk met de import statements.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import scipy.stats as stats

## Data en data dictionary

Hieronder lezen we de dataset in en tonen we enkele rijen.

In [2]:
ames_housing_data = pd.read_csv("ames_housing.csv")
# We werken met een subset van de dataset( die bevaat ongeveer 80) variabelen.
ames_housing_data = ames_housing_data[[
    "GrLivArea", "LotShape", "ExterQual", "OverallQual", "Neighborhood", "TotRmsAbvGrd", "SalePrice"]]
ames_housing_data

,GrLivArea,LotShape,ExterQual,OverallQual,Neighborhood,TotRmsAbvGrd,SalePrice
0,1710,Reg,Gd,7,CollgCr,8,208500
1,1262,Reg,TA,6,Veenker,6,181500
2,1786,IR1,Gd,7,CollgCr,6,223500
3,1717,IR1,TA,7,Crawfor,7,140000
4,2198,IR1,Gd,8,NoRidge,9,250000
...,...,...,...,...,...,...,...
1455,1647,Reg,TA,6,Gilbert,7,175000
1456,2073,Reg,TA,6,NWAmes,7,210000
1457,2340,Reg,Ex,7,Crawfor,9,266500
1458,1078,Reg,TA,5,NAmes,5,142125


Op grond van het bekijken van de inhoud in bovenstaand dataframe en met behulp van de informatie op [Kaggle](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data) kunnen we komen de volgende data dictionary.

| Variabele naam | Datatype | Numeriek/Categorisch | Soort variabele | Toelichting                                                                               |
|----|----------|----------------------|------|-------------------------------------------------------------------------------------------|
| GrLivArea | Integer  | Numeriek             | Ratio | De oppervlakte van het huis.                                                              |
| LotShape | String   | Categorisch          | Ordinaal | De vorm van het huis.                                                                     |
| ExterQual | String   | Categorisch          | Ordinaal | De kwaliteit van de buitenkant van het huis.                                              |
| OverallQual | Integer  | Categorisch          | Ordinaal | De overall kwaliteit van het huis.                                      |
| Neighborhood | String   | Categorisch          | Nominaal | De buurt waar het huis staat. |
| TotRmsAbvGrd | Integer  | Numeriek             | Ratio | Het aantal kamers boven de grond (exclusief badkamers). |
| SalePrice | Integer  | Numeriek             | Ratio | De verkoopprijs. |

Merk op dat we OverallQual zien als categorisch ondanks het Integer datatype. Ik zie de cijfers als labels, niet als getal. We controleren welke datatypes pandas heeft toegekend.

In [3]:
ames_housing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   GrLivArea     1460 non-null   int64 
 1   LotShape      1460 non-null   object
 2   ExterQual     1460 non-null   object
 3   OverallQual   1460 non-null   int64 
 4   Neighborhood  1460 non-null   object
 5   TotRmsAbvGrd  1460 non-null   int64 
 6   SalePrice     1460 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 80.0+ KB


Dat ziet er goed uit. Omdat we GrLivArea, TotRmsAbvGrd en SalePrice willen zien als ratio (vanwege het nulpunt) zouden we ze kunnen converteren naar float. Maar dat is niet echt nodig voor de analyse. Ook OverallQual zouden we kunnen converteren naar String en ook dat is niet echt nodig.

## Afzonderlijke variabelen

We gaan nu de afzonderlijke variabelen bekijken. Het is daarbij handig een onderscheid te maken tussen categorische en numerieke variabelen. Het is een goede gewoonte om een lijstje te maken met de namen van de categorische variabelen en de numerieke variabelen.

In [4]:
cat_vars = ["LotShape", "ExterQual", "OverallQual", "Neighborhood"]
num_vars = ["GrLivArea", "TotRmsAbvGrd", "SalePrice"]

### Categorische variabelen

Nu bekijken we stuk voor stuk de categorische variabelen.

#### LotShape

LotShape zegt iets over de vorm van het huis. We zijn geïnteresseerd welke mogelijke waarden LotShape kan hebben en hoe vaak die voorkomen.

In [5]:
ames_housing_data["LotShape"].value_counts()

Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64

Wat opvalt is dat de meeste huizen de vorm 'Reg' of 'IR1' hebben en maar heel weinig 'IR2' of 'IR3'. Met behulp van de informatie op [Kaggle](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data) kunnen we ook het volgende zeggen over de betekenis van de waarden:

       Reg	Regular
       IR1	Slightly irregular
       IR2	Moderately Irregular
       IR3	Irregular
Dat ziet eruit zoals we willen. Nu bekijken we de LotShape in een frequentieverdeling.

In [11]:
sns.countplot(ames_housing_data, x = "LotShape")

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

We zien hier opnieuw de dat de meeste huizen Reg of IR1 zijn.

#### ExterQual

Ook voor ExterQual, de kwaliteit van de buitenkant van het huis, bekijken we de mogelijke waarden.

In [12]:
ames_housing_data["ExterQual"].value_counts()

TA    906
Gd    488
Ex     52
Fa     14
Name: ExterQual, dtype: int64

We gebruiken weer de toelichting op Kaggle om deze waarden beter te begrijpen:

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor

Merk op dat Po (Poor) niet in onze data voorkomt.

We maken ook een frequentieverdeling.

In [13]:
sns.countplot(ames_housing_data, x="ExterQual")

C:\ProgramData\Anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

Bovenstaande grafiek willen we verbeteren. Want we willen dat de waarden op de goede volgorde staan, van slechte kwaliteit naar goede kwaliteit.

In [14]:
ExterQual_order = ["Po", "Fa", "TA", "Gd", "Ex"]
sns.countplot(ames_housing_data, x="ExterQual", order=ExterQual_order)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

Tenslotte, het belangrijkste, de interpretatie: Wat opvalt is dat de meeste huizen de kwaliteit Average/Typical (TA) of Good (Gd) hebben.

#### OverallQual

We bekijken weer de mogelijke waarden voor OverallQual (de overall kwaliteit van het huis).

In [15]:
ames_housing_data["OverallQual"].value_counts()

5     397
6     374
7     319
8     168
4     116
9      43
3      20
10     18
2       3
1       2
Name: OverallQual, dtype: int64

Merk op dat de categorische ordinale variabele is weergegeven met integers. Op Kaggle vinden we een toelichting:

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor

Het belangrijkste lijkt me: hoe hoger het getal, hoe beter de kwaliteit.

We kijken ook weer naar de frequentieverdeling.

In [16]:
sns.countplot(ames_housing_data, x="OverallQual")

C:\ProgramData\Anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

Als je bovenstaande figuur bekijkt dan heeft het wel iets weg van een normale verdeling.

#### Neighborhood

De Neighborhood is de buurt waar het huis staat. We kijken keer naar mogelijke waarden en hun voorkomens.

In [ ]:
ames_housing_data["Neighborhood"].value_counts()

We kunnen het ook weer visueel maken.

In [ ]:
ax = sns.countplot(ames_housing_data, x="Neighborhood")
ax.tick_params(axis='x', rotation=90) # Om ervoor te zorgen dat de labels leesbaar zijn.

Eigenlijk vind ik bovenstaande figuur niet echt veelzeggend. Mooier zou het zijn om een heatmap te maken op een geografische kaart van Boston waar deze wijken zijn. Maar dat vraagt wat uitzoekwerk dat ik nu even laat voor wat het is.

Wel kan het prettig zijn om te sorteren op frequentie. Dan is een horizontaal staafdiagram vaak ook wat geschikter.

In [ ]:
sns.countplot(ames_housing_data,
              y="Neighborhood",
              # Je zou verwachten dat sns.countplot kan sorteren op count, maar dat is niet zo.
              # We maken er gebruik van data value_counts() wel sorteert. Er zijn ook andere manieren
              # om dit op te lossen (bijvoorbeeld groupby op het dataframe aanroepen).
              order=ames_housing_data["Neighborhood"].value_counts().index)

Als er erg veel waarden zijn (hier valt het wel mee) kan het ook interessant zijn om naar een top N te kijken. Bijvoorbeeld naar de top 5.

In [ ]:
sns.countplot(ames_housing_data,
              y="Neighborhood",
              # We doen hetzelfde als in de vorige cell, maar nu voegen we head(5) toe
              # om de vijf bovenste (en dus grootste) waarden te krijgen.
              order=ames_housing_data["Neighborhood"].value_counts().head(5).index)

Hierboven zien we dus de vijf wijken met het grootste aantal huizen in de dataset.

### Numerieke variabelen

Je kunt gebruik maken van describe() om in één keer informatie te krijgen over alle numerieke variabelen in het dataframe.

In [ ]:
ames_housing_data[num_vars].describe()

We kunnen best wel het een en ander zeggen over bovenstaande tabel. Maar is het vaak fijner om te kijken naar histogrammen van de afzonderlijke variabelen. Want dan wordt duidelijker zichtbaar hoe de verdeling van de waarden is.

#### SalePrice

SalePrice is de verkoopprijs van het huis. Hieronder zie je het bijbehorende histogram.

In [ ]:
sns.histplot(ames_housing_data, x="SalePrice")

We zien hier een zogenaamde rechts-scheve verdeling (of right tail distribution). Want aan de rechterkant van de modus (en ook wel de mean) zijn meer waarden dan aan de linkerkant.

Voor sommige data-analyses, en met name machine learning, kan het helpend zijn om de verdeling te transformeren met een logaritmische functie om een meer normale verdeling te krijgen.

In [ ]:
ames_housing_data["SalePriceLog"] = np.log(ames_housing_data["SalePrice"])
sns.histplot(ames_housing_data, x="SalePriceLog")

Nice! Dat ziet er meer uit als een normale verdeling.

Als je terugkijkt naar het oorspronkelijke histogram van de SalePrice dan valt nog iets anders op. Er lijken namelijk best wat uitbijters (outliers) te zijn. Laten we die eens wat nader bekijken. We doen dat door naar de top 10 hoogste prijzen te kijken.


In [ ]:
ames_housing_data.nlargest(10, columns="SalePrice")

Wat hier opvalt: als het gaat over huizen met prijzen boven de 500.000 dollar dan is er maar weinig data om mee te werken. Dus als je deze data bijvoorbeeld zou willen gebruiken om met machine learning de prijs van een huis te voorspellen, dan presteert dat model mogelijk niet zo goed voor dure huizen.

#### GrLivArea

GrLivArea is het oppervlakte van het huis. We bekijken weer het histogram.

In [ ]:
sns.histplot(ames_housing_data, x="GrLivArea")

Ook hier weer een rechts-scheve verdeling en een aantal forse outliers. Laten we eerst naar de outliers kijken.

In [ ]:
ames_housing_data.nlargest(10, columns="GrLivArea")

Je ziet dat er maar een klein aantal huizen zijn boven de 4000 square feet. Je zou misschien verwachten dat het dezelfde huizen zijn als de dure huizen. Maar dat blijkt maar gedeeltelijk zo te zijn. Het grootste huis is juist erg goedkoop! (160.000 dollar). Dat is misschien een wel erg bijzonder geval in de data!

Ook voor de GrLivArea kijken we of een log transform de data wat meer normaliseert.

In [ ]:
ames_housing_data["GrLivAreaLog"] = np.log(ames_housing_data["GrLivArea"])
sns.histplot(ames_housing_data, x="GrLivAreaLog")

Ook hier zien we dat de data iets meer normaal verdeeld lijkt.

#### TotRmsAbvGrd

TotRmsAbvGrd tenslotte is het aantal kamers boven de grond exclusief badkamers. De variabele is als volgt verdeeld.

In [ ]:
sns.histplot(ames_housing_data, x="TotRmsAbvGrd")

Dit ziet er behoorlijk normaal verdeeld uit. Misschien een heel klein beetje scheef naar rechts.

We kunnen overigens ook de scheefheid van de verdeling in een getal uitdrukken. We doen dat voor de SalePrice, GrLivArea en TotRmsAbvGrd zodat we ze kunnen vergelijken. Zie ook: [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.skew.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.skew.html).

In [ ]:
ames_housing_data["SalePrice"].skew(), ames_housing_data["GrLivArea"].skew(), ames_housing_data["TotRmsAbvGrd"].skew()

Hoe groter het getal hoe schever naar rechts. Als je kijkt naar de verschillen tussen de drie getallen hierboven dan komt dat ook wel overeen met wat we zien in de histogrammen. SalePrice is het meest scheef, TotRmsAbvGrd het minst.

Het is misschien ook leuk om te kijken naar de scheefheid van de getransformeerde kolommen.

In [ ]:
ames_housing_data["SalePriceLog"].skew(), ames_housing_data["GrLivAreaLog"].skew()

Zoals verwacht liggen de waarden veel dichter bij de nul.

## Relaties tussen de variabelen

We gaan nu kijken naar de relaties tussen de variabelen. We doen dat eerst systematisch door alle mogelijke tweetallen te bekijken. Daarbij is het nodig onderscheid te maken tussen een tweetal van numerieke variabelen, een tweetal van categorische variabelen, en een tweetal met een numerieke en een categorische variabelen. Vervolgens kijken we ook nog naar de mogelijkheid om naar de relatie tussen drie variabelen te kijken.

### Tweetallen van numerieke variabelen.

Voor tweetallen van numerieke variabelen kun je makkelijk een overzicht maken van alle mogelijke combinaties door een correlatiematrix en een pairplot te maken. Laten we beginnen met een pairplot.

In [ ]:
sns.pairplot(ames_housing_data[num_vars])

Een pairplot maakt voor iedere combinatie van numerieke variabelen een scatterplot. Omdat het niet zo interessant is om een variabelen met zichzelf te vergelijken staan op de diagonaal histogrammen.

Als je kijkt naar bovenstaande figuur dan lijken er best wat afhankelijkheden te zijn tussen de variabelen. Bijvoorbeeld tussen de GrLivArea en de SalePrice. Dat zou je ook wel verwachten op grond van de betekenis van de variabelen. Hoe groter het huis, hoe duurder. We kunnen de afhankelijkheid uitdrukken met de correlatie coëfficiënt.

Hieronder zetten we ze in één keer in een tabel.

In [ ]:
corr_matrix = ames_housing_data[num_vars].corr()
corr_matrix

Je ziet inderdaad een aantal hoge coëfficiënten. Zelf vind ik de correlatie tussen TotRmsAbvGrd en SalePrice wat lager dan ik zou gokken.

Nu zoomen we in op in één tweetal: GrLivArea en SalePrice. Die hebben naar verwachtiging een grote correlatie. We bestuderen die apart in een regplot. Dat is een scatterplot met daarin een regressielijn getekend.

In [ ]:
sns.regplot(ames_housing_data, x="GrLivArea", y="SalePrice")

Wat opvalt, naast de regressielijn die regplot heeft getekend, is dat de puntenwolk kegelvormig is. Dat kan komen doordat beide variabelen rechs-scheef verdeeld zijn. Laten we eens kijken naar een regplot van de getransformeerde kolommen.

In [ ]:
sns.regplot(ames_housing_data, x="GrLivAreaLog", y="SalePriceLog")

Mooi! Je ziet dat de punten dichter bij de regressielijn lijken te zitten. Het is mogelijk om een getal te geven aan de voorspellende kwaliteit van de regressielijn, namelijk de $R^2$. Voor twee variabelen is dat de correlatie coëfficiënt in het kwadraat.

In [ ]:
r2 = ames_housing_data["GrLivArea"].corr(ames_housing_data["SalePrice"])**2
r2_log_vars = ames_housing_data["GrLivAreaLog"].corr(ames_housing_data["SalePriceLog"])**2
r2, r2_log_vars

Dat betekent dat de SalePrice ongeveer voor de helft wordt verklaard door de GrLivArea. Bij de log variabelen is de $R^2$ net iets hoger. Dus dan is de voorspellende/verklarende kwaliteit net wat beter.

### Tweetallen van categorische variabelen

De tweetallen van categorische variabelen bekijken we stuk voor stuk. Er zijn zes mogelijke combinaties. We kijken iedere keer naar een gelabeld staafdiagram.

#### LotShape en ExterQual

In [ ]:
# We gebruiken de lijst ExterQual_order die we eerder hebben gemaakt om de labels
# op de goede volgorde zetten. Voor LotShape is het niet nodig omdat de alfobetische
# volgorde al de goede volgorde is.
sns.countplot(ames_housing_data, x="LotShape", hue="ExterQual", hue_order=ExterQual_order)

We kijken vooral naar Reg en IR1, omdat IR2 en IR3 maar heel weinig voorkomens hebben. Wat dan opvalt is dat huizen met een (licht) onregelmatige vorm een iets betere buitenkwaliteit hebben dan huizen met een regelmatige vorm.

Het is mogelijk om de afhankelijkheid van LotShape en ExterQual uit te drukken in getallen. We doen daarvoor een $\chi^2$ test. Het is dan wel aardig om eerst de kruistabel te bekijken.

In [ ]:
crosstab = pd.crosstab(ames_housing_data["LotShape"], ames_housing_data["ExterQual"])
crosstab

Je ziet dat er frequenties onder de 5 voorkomen. Dat 'mag niet' bij een $\chi^2$ test. We kunnen dat hier oplossen door IR1, IR2 en IR3 samen te nemen en door Fa en TA samen te nemen.

In [ ]:
LotShape_mapper = {"IR1": "Irr", "IR2": "Irr", "IR3": "Irr", "Reg": "Reg"}
EXterQuall_mapper = {"Fa": "FaTA", "TA": "FaTA", "Gd": "Gd", "Ex": "Ex"}
crosstab = pd.crosstab(ames_housing_data["LotShape"].map(LotShape_mapper),
                       ames_housing_data["ExterQual"].map(EXterQuall_mapper))
crosstab

Nu kunnen we de $\chi^2$ test doen.

In [ ]:
results = stats.chi2_contingency(crosstab)
"Chi kwadraat = {}, p-waarde = {}".format(results[0], results[1])

Dat is een zeer klein p-waarde. Er is dus waarschijnlijk sprake van afhankelijkheid. Voor de volgende combinaties laten we de $\chi^2$ test voor wat die is en doen we de interpretatie 'op het oog'.

#### LotShape en OverallQual

In [ ]:
sns.countplot(ames_housing_data, x="LotShape", hue="OverallQual")

We zien hetzelfde bij de OverallQual als bij de ExterQual. De kwaliteit van de IR1 huizen lijkt iets hoger te zijn.

#### Lotshape en Neighborhood

We bekijken alleen de top 10 grootste buurten om de grafiek leesbaar te houden.

In [ ]:
sns.countplot(ames_housing_data, x="LotShape", hue="Neighborhood",
              hue_order=ames_housing_data["Neighborhood"].value_counts().head(10).index)

Opvallend hier is de vorm van huizen in de wijk Gilbert. De grafiek suggereert dat daar veel meer dan in de andere wijken huizen zijn met niet-regelmatige vormen. Een [google search](https://www.google.com/search?q=boston+gilbert+neighborhood) op deze wijk bevestigt het beeld dat bovenstaande grafiek geeft: huizen met bijzondere vormen.

Bijvoorbeeld:

![huis](https://photos.zillowstatic.com/fp/25cd42d3fd563d13eafcba923d40543f-cc_ft_1536.jpg)

Bron: [https://photos.zillowstatic.com/fp/25cd42d3fd563d13eafcba923d40543f-cc_ft_1536.jpg](https://photos.zillowstatic.com/fp/25cd42d3fd563d13eafcba923d40543f-cc_ft_1536.jpg)

#### ExterQual en OverallQual

We verwachten op voorhand een grote afhankelijkheid tussen ExterQual en OverallQual. Laten we eens kijken of dat inderdaad zo is.

In [ ]:
sns.countplot(ames_housing_data, x="OverallQual", hue="ExterQual", hue_order=ExterQual_order)

Ja, je ziet duideljk dat TA (Average/Typical) meer verdeeld is rond de 5 en dat Gd(Good) meer verdeeld is rond de 7.

#### ExterQual en Neighborhood

In [ ]:
ax = sns.countplot(ames_housing_data, x="Neighborhood", hue="ExterQual",
              hue_order=ExterQual_order,
              order=ames_housing_data["Neighborhood"].value_counts().head(10).index)
ax.tick_params(axis='x', rotation=90) # Om ervoor te zorgen dat de labels leesbaar zijn.

Ook hier is een afhankelijkheid te zien. Zoals je misschien ook zou verwachten verschilt de kwaliteit per wijk. In de wijk NridgHt (Northridge Heights) bijvoorbeeld is de kwaliteit een stuk beter dan in andere wijken.

#### OverallQual en Neighborhood

We verwachten dan bij OverallQual en Neighborhood ongeveer hetzelfde patroon te zien. Want OverallQual en ExterQuall zijn afhankelijk van elkaar. We bekijken maar zeven wijken vanwege de leesbaarheid.

In [ ]:
ax = sns.countplot(ames_housing_data, x="Neighborhood", hue="OverallQual",
              order=ames_housing_data["Neighborhood"].value_counts().head(7).index)
ax.tick_params(axis='x', rotation=90) # Om ervoor te zorgen dat de labels leesbaar zijn.

We zien inderdaad dezelfde verschillen terugkomen, bijvoorbeeld weer in NridgHt (Northridge Heights).

### Tweetallen met een numerieke en een categorische waarde

Als we een numerieke en een categorische variabele met elkaar vergelijken, dan zijn we geïnteresseerd in de verdelingen van de numerieke variabele per waarde die de categorische variabele aan kan nemen. De boxplot is daar erg geschikt voor.

#### LotShape en GrLivArea

In [ ]:
sns.boxplot(ames_housing_data, x="LotShape", y="GrLivArea")

De verdelingen lijken best dicht bij elkaar te liggen. Aan de andere kan zijn er ook duidelijk verschillen zichtbaar. Maar wat is 'dicht bij elkaar' en wat is verschillend? Daarvoor kun je een statistische analyse doen. Bijvoorbeeld een ANOVA test om te kijken of de verdelingen significant verschillen. Maar ANOVA vereist wel dat de numerieke variabele normaal verdeeld is. In ons geval is dat GrLivArea en die is niet normaal verdeeld, maar rechts-scheef. We hebben nu twee opties:
- De ANOVA test op de log transformatie die wel normaal verdeeld is;
- Een andere test doen die om kan gaan met niet normaal verdeelde variabelen, namelijk de Kruskal-Wallis test.

We doen het allebei.

In [ ]:
# Ik maak hier gebruik van de gelegenheid om met list comprehensions
# te werken. Dat is niet per se nodig. Uiteindelijk wil je de functies
# stats.anova en stats.kruskal de goede argumenten kunnen geven. Dat kan op veel
# verschillende manieren. List comprehensions maken het compact en leesbaar, maar
# zijn vaak wel wat lastiger om 'uit het niks' zelf te bedenken.

# Eerst delen we het dataframe op in groepen.
# Voor iedere waarde van LotShape is er een groep.
groups = [ames_housing_data[ames_housing_data["LotShape"] == val]
          for val in ames_housing_data["LotShape"].unique()]

# Voor ANAVO gebruiken we de GrLivAreaLog en voor Kruskal-Wallis GrLivArea
groups_anova = [group["GrLivAreaLog"] for group in groups]
groups_kruskal = [group["GrLivArea"] for group in groups]

# Nu kunnen we de testen doen met de bijbehorende library functies uit scipy.stats.
# Met * unpack je een lijst naar losse argumenten.
stat_anova, p_anova = stats.f_oneway(*groups_anova)
stat_kruskal, p_kruskal = stats.kruskal(*groups_kruskal)

# We zijn vooral geinteresseerd in de p-waarde.
"P-waardes: ANOVA: {}, Kruskal-Wallis: {}".format(p_anova, p_kruskal)

Je ziet: kleine p-waardes. Dus de variabelen zijn waarschijnlijk afhankelijk.

We kunnen ook een andere aanpak kiezen. We maken dan gebruik van het feit dat LotShape ordinaal is. We kunnen die variabele coderen als integer en de gecodeerde variabele dan als numeriek behandelen.

We doen eerst de codering.

In [ ]:
# We maken eerst een encoder die de vertaling LotShape naar integer definieert.
LotShape_encoder = {"Reg": 1, "IR1": 2, "IR2": 3, "IR3": 4}
# Die kunnen we gebruiken om de codering in een nieuwe kolom te zetten.
ames_housing_data["LotShapeCode"] = ames_housing_data["LotShape"].map(LotShape_encoder)

ames_housing_data[["LotShape", "LotShapeCode"]]

Ziet er goed uit. Nu gaan we LotShapeCode vergelijken met de GrLivArea. Omdat LotShape een ordinale variabele is, moet je een ander soort correlatie berekenen. Namelijk Spearman correlatie.

In [ ]:
ames_housing_data["LotShapeCode"].corr(ames_housing_data["GrLivArea"], method="spearman")

Dat is een best lage correlatie coëfficient. Wat is nu waar? We weten het niet precies! Correlatie coëfficienten berekenen op ordinale variabelen is eigenlijk een beetje valsspelen. Maar ANOVA en Kruskal-Wallis leveren heel vaak hele lage p-waarden op. Gevoelsmatig ligt de correlatie coëfficient wat dichterbij de 'afhankelijkheid op het oog' dan de extreem lage p-waarden van Kruskal-Wallis en ANOVA.

#### LotShape en SalePrice


In [ ]:
sns.boxplot(ames_housing_data, x="LotShape", y="SalePrice")

Het beeld van de grafiek lijkt wat anders te zijn bij Reg en IR1. Je ziet daar namelijk duidelijk een rechts-scheve verdeling. Die bij IR2 en zeker IR3 minder is te zien. Maar dat komt vooral omdat Reg en IR1 veel meer voorkomens hebben dan IR2 en IR3. Met een ander grafiektype is dat wat beter te zien.

In [ ]:
sns.stripplot(ames_housing_data, x="LotShape", y="SalePrice")

Ook hier geldt dus weer: er lijkt een lichte afhankelijkheid te zijn tussen LotShape en SalePrice. Ook hier zouden we een statistiche analyse kunnen doen, maar we laten het voor wat het is.

#### ExterQual en GrLivArea

In [ ]:
sns.boxplot(ames_housing_data, x="ExterQual", y="GrLivArea", order=ExterQual_order)

Hier zien we een duidelijk verband: hoe groter het huis, hoe beter de kwaliteit van de buitenkant. Je kunt daar allerlei mogelijke verklaringen voor bedenken, bijvoorbeeld: Mensen met een groot huis hebben meer geld en daardoor ook meer geld beschikbaar voor onderhoud.

#### ExterQual en SalePrice

Voor de kwaliteit en de verkoopprijs verwachten we natuurlijk een verband.

In [ ]:
sns.boxplot(ames_housing_data, x="ExterQual", y="SalePrice", order=ExterQual_order)

En jawel, hoe beter de kwaliteit hoe hoger de prijs.

#### OverallQual en de numerieke variabelen

Er is een sterke afhankelijkheid tussen ExterQual en OverallQual. Dus we verwachten dezelfde verbanden te zien. Laten we eens kijken.

In [ ]:
sns.boxplot(ames_housing_data, x="OverallQual", y="GrLivArea")

In [ ]:
sns.boxplot(ames_housing_data, x="OverallQual", y="SalePrice")


De twee bovenstaande grafieken zijn inderdaad zoals verwacht.

#### Neighborhood en GrLivArea

In [ ]:
ax = sns.boxplot(ames_housing_data, x="Neighborhood", y="GrLivArea")
ax.tick_params(axis='x', rotation=90)

Je ziet behoorlijke verschillen. Het kan wel fijn zijn om de wijken te sorteren op de mean van de GrLivArea.

In [ ]:
ax = sns.boxplot(ames_housing_data, x="Neighborhood", y="GrLivArea",
                 order=ames_housing_data.groupby(["Neighborhood"])["GrLivArea"].median().sort_values().index)
ax.tick_params(axis='x', rotation=90)

Ook nu zien we inderdaad dat het woonoppervlakte verschilt per wijk. Dat is ook wel in lijn met wat je zou verwachten.

#### NeighborHood en SalePrice

In [ ]:
ax = sns.boxplot(ames_housing_data, x="Neighborhood", y="SalePrice",
                 order=ames_housing_data.groupby(["Neighborhood"])["SalePrice"].median().sort_values().index)
ax.tick_params(axis='x', rotation=90)

Zoals te verwachten was: de verkoopprijs van huizen lijkt afhankelijk te zijn van de wijk.

### Drietallen van variabelen

Soms wil je ook de relatie zien tussen drie variabelen. Er zijn veel verschillende combinaties. Hier bekijken we er twee.

#### SalePrice, GrLivArea, ExterQual

We kijken naar de relatie tussen de SalePrice, GrLivArea en de LotShape. Dat zijn twee numerieke variabelen en één categorische. Dat kun je als volgt visueel maken. We gebruiken de log transformaties. Daar worden de grafieken beter leesbaar van.

In [ ]:
sns.scatterplot(ames_housing_data, x="GrLivAreaLog", y="SalePriceLog", hue="ExterQual")

Hier zie je als het ware clusters van huizen met dezelfde kwaliteit. Je kunt een categorische variable ook in één keer met alle numerieke variabelen vergelijken met een pairplot. Op de diagonaal zie je dan de vergelijking met een numerieke variabele. In de overige vakken de vergelijking met alle mogelijke tweetallen.

In [ ]:
sns.pairplot(ames_housing_data[num_vars + ["LotShape"]], hue="LotShape")

Bovenstaande grafiek geeft ons niet zoveel inzicht. In het algemeen is een grafiek als deze bruikbaar als je specifiek geïnteresseerd bent in één categorische variabele, met name als je die wilt voorspellen met een classificatie model.

#### LotShape, ExterQuall, SalePrice

Nu kijken we naar twee categorische variabelen (LotShape en ExterQuall) en een numerieke (SalePrice). Het is dan helpend om de data te aggregeren.

In [ ]:
mean_SalePrice = ames_housing_data.groupby(['LotShape', 'ExterQual'])['SalePrice'].mean()
mean_SalePrice

Nuttig, maar we willen dit liever zien in een grafiek.

In [ ]:
# Merk op dat we bovenstaande groupby tabel niet nodig hebben.
# Dat doet seaborn voor ons.
sns.catplot(data=ames_housing_data, x='LotShape', y='SalePrice',
            hue='ExterQual', hue_order=ExterQual_order,
            kind='bar', errorbar=None)

Wat hier opvalt is dat bij Reg, IR1, IR2 een zelfde patroon is te zien in de relatie SalePrice en ExterQual. Maar dat patroon wordt gebroken bij IR3. Echter, neem in herinnering dat er maar 10 rijen zijn met IR3. Hier kunnen we dus geen conclusies uit trekken.

## Conclusie

We hebben alle variabelen afzonderlijk bekeken en ook alle mogelijke tweetallen van variabelen met elkaar vergeleken. Dat heeft een aantal inzichten gegeven. Bijvoorbeeld dat de huizenprijs verschilt per wijk (zoals je zou verwachten). Dat soort inzichten kunnen ons helpen als we meer gericht met de data aan de slag gaan. Bijvoorbeeld als we met machine learning een voorspelling zouden willen doen van de huizenprijs. Een interessante vraag daarbij is in welke mate de variabelen gezamenlijk de huizenprijs beter kunnen voorspellen dan we hier hebben gezien bij de analyse van tweetallen.